In [2]:
import json
import pandas as pd
import re
import csv
import copy
import nltk
import stemming
from nltk.stem import PorterStemmer
from stemming.porter2 import stem

In [3]:
#PA review with business_id and date and stars and text
data = [] 
cf = open('PAreview.csv','r', encoding='utf-8')

file = csv.DictReader(cf)

for x in file:
    line = {'business_id':x['business_id'],'date':x['date'],'stars':x['stars'],'text':x['text']}
    data.append(line)

cf.close()


In [3]:
data_5star=[]
data_4star=[]
data_3star=[]
data_2star=[]
data_1star=[]
for i in range(len(data)):
    if data[i]['stars']=='5.0':
        data_5star.append(data[i])
    elif data[i]['stars']=='4.0':
        data_4star.append(data[i])
    elif data[i]['stars']=='3.0':
        data_3star.append(data[i])
    elif data[i]['stars']=='2.0':
        data_2star.append(data[i])
    else:
        data_1star.append(data[i])


In [4]:
len(data_5star)

81376

In [5]:
len(data_4star)

60189

In [6]:
len(data_3star)

30048

In [7]:
len(data_2star)

20190

In [8]:
len(data_1star)

22640

In [24]:
reviewslist = copy.deepcopy(data)
#reviewslist

[{'business_id': 'eU_713ec6fTGNO4BegRaww',
  'date': '2013-01-20 13:25:59',
  'stars': '4.0',
  'text': 'I\'ll be the first to admit that I was not excited about going to La Tavolta. Being a food snob, when a group of friends suggested we go for dinner I looked online at the menu and to me there was nothing special and it seemed overpriced.  Im also not big on ordering pasta when I go out. Alas, I was outnumbered. Thank goodness! I ordered the sea bass special. It was to die for. Cooked perfectly, seasoned perfectly, perfect portion. I can not say enough good things about this dish. When the server asked how it was he seemed very proud of the dish and said, " doesn\'t she (the chef) do an incredible job?" She does. \n\nMy hubby got the crab tortellini and also loved his. I heard "mmmm this is so good" from all around the table. Our waiter was super nice and even gave us free desserts because we were some of the last people in the restaurant. Service was very slow and the place was PACK

In [25]:
REPLACE_NO_SPACE = re.compile("(\.)|(\;)|(\:)|(\!)|(\')|(\?)|(\,)|(\")|(\()|(\))|(\[)|(\])|(\$)|(\*)|(\%)|(\_)|(\=)|(\#)|(\&)|(\~)|(\@)")#[^\P{P}-]+
REPLACE_WITH_SPACE = re.compile("(\n)|(\-)|(\/)|(\d)")

def preprocess_reviews(reviews):
    reviews = REPLACE_NO_SPACE.sub("", reviews)
    reviews = REPLACE_WITH_SPACE.sub(" ", reviews)
    return reviews

reviewsTEXT_clean = copy.deepcopy(reviewslist)

#Stop words
stop_words = {'who', 'how', 'him', 'can', 'than', 'these', 'your', 'the', 'while', 'don', 'of', 'on', 'had', 'there', "you've", 'that', 'having', 'himself', "mustn't", 'same', 'are', "won't", 'then', 'itself', 'doing', 'from', 'both', 'where', 'wouldn', 'me', 'off', 'because', 'isn', "you'd", 'whom', 'mustn', 'is', 'themselves', 'no', 'very', 'up', 'd', 'ma', 'yours', 'been', 'ain', 'will', 'a', 'most', 'did', 'with', 'o', 'this', 'during', 'i', "mightn't", "isn't", 'being', 'couldn', 'them', 'not', 'such', 'her', 'some', 'only', "didn't", 'should', 'after', 'our', 'down', 'here', 'about', 'herself', "hadn't", 'but', 'he', 'an', 'am', 't', 'they', 'again', 'll', 've', 'didn', 'into', 'needn', 're', 'nor', "couldn't", 'above', 'all', "should've", 'm', 'other', 'below', "she's", 'just', 'between', 'hasn', 'own', 'yourselves', 'until', 'too', 'which', "shouldn't", 's', "it's", 'his', 'y', 'to', 'over', 'hadn', "shan't", 'does', 'weren', 'shouldn', 'under', "aren't", 'be', "don't", 'any', 'or', "haven't", 'she', 'aren', 'against', 'we', 'in', 'ourselves', 'have', 'won', "wasn't", 'wasn', 'you', 'what', 'mightn', "weren't", 'doesn', 'hers', 'myself', 'shan', 'before', 'more', "wouldn't", 'were', 'each', "doesn't", 'through', 'for', "hasn't", 'by', 'now', 'do', 'has', 'those', 'few', "you'll", 'once', 'it', 'their', 'further', "you're", 'my', 'at', 'when', 'yourself', 'why', 'as', 'was', 'and', 'out', "needn't", 'if', 'haven', 'its', 'theirs', "that'll", 'so', 'ours'}
negWords={'not',"wouldn't",'isn','wasn',"weren't", "haven't", "hasn't", "couldn't", "isn't", 'doesn','hasn',"mustn't", 'mightn', 'shan', 'no', "wasn't",'aren', "didn't", "hadn't","don't",'nor',"won't",'weren',"doesn't","needn't", 'shouldn',"mightn't","shan't", 'wouldn',"shouldn't",'hadn'}
ps = PorterStemmer()
stop_words_stem=[]
negWords_stem=[]
for w in stop_words:
    stop_words_stem.append(ps.stem(w))
for w in negWords:
    negWords_stem.append(ps.stem(w))
d=set(['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z','à','{sigh}'])
StopWords=set(stop_words)
StopWords.update(set(stop_words_stem))
StopWords=set(StopWords)-set(negWords)
StopWords=set(StopWords)-set(negWords_stem)
StopWords.update(d)

#Positive and Negative words
good_word = set(pd.read_csv('positive-words.txt',header=None,names=['word'])['word'])
bad_word = set(pd.read_csv('negative-words.txt',encoding="ISO-8859-1" ,header=None, names=['word'])['word'])
good_word, bad_word = good_word | set(stem(x) for x in good_word), bad_word | set(stem(x) for x in bad_word)
good_word, bad_word = good_word | set('NOT' + x for x in bad_word), bad_word | set('NOT' + x  for x in good_word)

for ind in range(len(reviewslist)):
    texts = ''
    texts = reviewslist[ind]['text']
    #Change into lower case
    texts = texts.lower()
    #Convert abbreviations into standard forms
    texts = re.sub('im','i am', texts)
    texts = re.sub('i\'m','i am', texts)
    texts = re.sub('\'s',' is', texts)
    texts = re.sub('\'re',' are', texts)
    texts = re.sub('\'ll',' will', texts) 
    texts = re.sub('\'d',' should', texts)  
    texts = re.sub('n\'t',' not', texts)
    texts = re.sub('isnt','isn\'t', texts)
    texts = re.sub('wasnt','wasn\'t', texts)
    texts = re.sub('werent','weren\'t', texts)
    texts = re.sub('dont','don\'t', texts)
    texts = re.sub('doesnt','doesn\'t', texts)
    texts = re.sub('didnt','didn\'t', texts)
    texts = re.sub('hasnt','hasn\'t', texts)
    texts = re.sub('havent','haven\'t', texts)
    texts = re.sub('hadnt','hadn\'t', texts)
    texts = re.sub('mightnt','mightn\'t', texts)
    texts = re.sub('shouldnt','shouldn\'t', texts)
    texts = re.sub('isn','isn\'t', texts)
    texts = re.sub('wasn','wasn\'t', texts)
    texts = re.sub('weren','weren\'t', texts)
    texts = re.sub('don','don\'t', texts)
    texts = re.sub('doesn','doesn\'t', texts)
    texts = re.sub('didn','didn\'t', texts)
    texts = re.sub('hasn','hasn\'t', texts)
    texts = re.sub('haven','haven\'t', texts)
    texts = re.sub('hadn','hadn\'t', texts)
    texts = re.sub('mightn','mightn\'t', texts)
    texts = re.sub('shouldn','shouldn\'t', texts)  
    texts = re.sub('n\'t',' not', texts)
    #print(texts)
    #Break paragraphs to sentences
    pattern = r'\.|\;|\!|\?|\,|\:|\'|\"|\%'
    list_text=re.split(pattern,texts)
    #print(list_text)
    sent=''
    for i in range(len(list_text)):
        list_text[i] = re.sub('\+','', list_text[i])
        list_text[i] = re.sub('\*','', list_text[i])
        list_text[i] = re.sub('\$','', list_text[i])
        list_text[i] = re.sub('\(','', list_text[i])
        list_text[i] = re.sub('\)','', list_text[i])
        list_text[i] = re.sub('\[','', list_text[i])
        list_text[i] = re.sub('\]','', list_text[i])
        list_text[i] = re.sub('\%','', list_text[i])
        list_text[i] = re.sub('\\\\',' ', list_text[i])
   
        
        #Remove stop words before normalization   
        wordList = list_text[i].split()
        wordList=[x for x in wordList if x not in StopWords]
        list_text[i] = " ".join(wordList) 
        #print(list_text[i])
            
        #add NOT mark after negative words
        matchObj1 = re.search(r'(.*)not (.*)',list_text[i])
        matchObj2 = re.search(r'(.*) never (.*)',list_text[i])
        
        if matchObj1 != None :
            sub=re.sub(r' ', " NOT", ' '+matchObj1.group(2))
            list_text[i] = re.sub(matchObj1.group(2) ,sub, list_text[i])
            list_text[i] = re.sub(r'not ','', list_text[i])
            sent = sent + list_text[i] + ' '
        
        elif matchObj2 != None :
            sub=re.sub(r' ', " NOT", ' '+matchObj2.group(2))
            list_text[i] = re.sub(matchObj2.group(2) ,sub, list_text[i])
            list_text[i] = re.sub(r'never ','', list_text[i])
            sent = sent + list_text[i] + ' '
        
        else:
            sent = sent + list_text[i] + ' '
            
    #print(sent)

    #Remove punctuations
    text_clean=preprocess_reviews(sent)

    #Normalize the words
    st = text_clean
    st2 = " ".join([stem(word) for word in st.split(" ")])
    text_clean = st2
    #print(text_clean)   
    
    #Remove stop words after normalization   
    wordList = text_clean.split()
    wordList=[x for x in wordList if x not in StopWords]
    
    #statistics of words
    reviewsTEXT_clean[ind]['text']=wordList
    reviewsTEXT_clean[ind]['good_num']= sum(word in good_word for word in wordList)
    reviewsTEXT_clean[ind]['bad_num'] = sum(word in bad_word for word in wordList)
    #reviewsTEXT_clean[ind]['freq']=nltk.FreqDist(reviewsTEXT_clean[ind]['text'])  
    #print([word in good_word for word in wordList])
    #print([word in bad_word for word in wordList])
    
    if ind%1000==0:
        print(ind)

#reviewsTEXT_clean

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000
135000
136000
137000
138000
139000
140000
141000
142000
143000
144000
145000
146000
147000
148000
149000
150000
151000
152000
153000
154000
155000
156000
157000
158000


In [26]:
reviewsTEXT_clean_df=pd.DataFrame.from_dict(reviewsTEXT_clean)
reviewsTEXT_clean_df.to_csv('PAreview_clean.csv')

In [27]:
review_attitude=reviewsTEXT_clean_df.loc[:,['business_id','bad_num','good_num']]
business_attitude=review_attitude.groupby(by=['business_id'])['bad_num','good_num'].sum()
business_attitude['business_id'] =business_attitude.index
business_attitude =business_attitude.reset_index(drop= True)

business = [] 
cf = open('PA_business_cleaning.csv','r', encoding='utf-8')

file = csv.DictReader(cf)

for x in file:
    line = {'business_id':x['business_id'],'stars':x['stars']}
    business.append(line)

cf.close()

business=pd.DataFrame.from_dict(business)
business=pd.merge(business,business_attitude)
business.to_csv('PABusinessGoodBadWords.csv')
